# Анализ выбора портфеля по коэффициенту Шарпа жадным алгоритмом

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from datetime import datetime

Считываем данные

In [26]:
start = datetime.now()
raw_data = pd.read_csv("Россия 37 Акций 2016-01.07.2019.txt")
raw_data = np.array(raw_data)
periods_in_data = raw_data.shape[0]
print(periods_in_data)

881


In [21]:
print(raw_data)

[[ 2.01601040e+07 -1.52180400e+00 -1.52155153e+00 ... -2.70986745e+00
   4.71380471e-01  1.11663653e+01]
 [ 2.01601050e+07  3.02138343e+00  1.12824001e+00 ...  3.70874962e+00
   2.37935657e+00  1.12647418e+01]
 [ 2.01601060e+07 -8.40336134e-02 -3.16966245e-01 ... -1.16771274e+00
  -5.72831424e-01  3.80116959e+00]
 ...
 [ 2.01906270e+07  4.01231688e-01  1.62874666e-01 ...  6.00200067e-01
   1.16510655e+00  1.03524229e+00]
 [ 2.01906280e+07 -1.39405204e+00 -4.96823352e-01 ... -4.97182632e-01
  -3.34678610e-01 -1.26444299e+00]
 [ 2.01907010e+07  1.38548539e+00  1.28785003e+00 ...  4.16389074e-01
  -6.81481481e-01  3.86398764e+00]]


In [27]:
data = np.delete(data, 0, axis = 1)
#data = np.delete(data, 0, axis = 1)  # Удалять дополительно второй столбец только в часовых котировках

stocks_to_portf_min = 5
stocks_to_portf_max = 6
stocks_to_portf_step = 1

sharp_period_min = 20
sharp_period_max = 320
sharp_period_step = 20

hold_period_min = 10       # Важна кратность диапазона end-start по hold(step)
hold_period_max = 190
hold_period_step = 10

sharp_period = 125
start_of_test_period = sharp_period_max
end_of_test_period = periods_in_data // hold_period_max * hold_period_max

mean_annual_profit_overal = []
P_L_overal = []
random_income_overal = []
random_portf_list = []
# Рассматриваем разное количество акций в портфеле по Кэф Шарпа
for stocks_to_portf in range(stocks_to_portf_min, stocks_to_portf_max, stocks_to_portf_step):
    # Рассматриваем разный период рассчета Кэф Шарпа.
    for sharp_period in range(sharp_period_min,sharp_period_max,sharp_period_step):
        #Смотрим также результаты стратегии Купи и держи в те же периоды.
        P_L_by_hold = []
        mean_annual_profit_by_hold = []
        random_income_hold = []
        for hold in range(hold_period_min,hold_period_max,hold_period_step):
            start_of_test_period = sharp_period_max # Должен быть больше чем sharp_period
            columns_count = data.shape[1] # число акций
            stocks_to_portf_list = []
            overal_results = []
            random_portf_list = []
            
            for i in range(start_of_test_period,end_of_test_period,hold):
            #for i in range(sharp_period,sharp_period+hold*10,hold):        # Пересчитываем через период hold коэфф Шарпа
                stocks_to_portf_list = []          
                counter = 1
                random_portf = np.mean(np.array(data[i-sharp_period:i,:]))
                random_array = np.array(data[i-sharp_period:i,:])
                random_portf_list.append(random_portf)
                for k in range(0, stocks_to_portf):
                    sharp_ratio_list = []
                    if counter == 1:                             # Выбираем первую акцию в портфель
                        for j in range(0, columns_count):
                            share_prices = np.array(data[i-sharp_period:i,j], dtype=float)
                            income = np.sum(share_prices)
                            std = np.std(share_prices)
                            if std > 0 and income > 0:
                                sharp_ratio = income/std
                            else:
                                sharp_ratio = 0
                            sharp_ratio_list.append(sharp_ratio)
                        
                        max_sharp_ratio = max(sharp_ratio_list)
                        indx_max_sharp_ratio = np.argmax(sharp_ratio_list)
                        comulative_income = np.array(data[i-sharp_period:i,indx_max_sharp_ratio], dtype=float)   # Инициализируем доход портфеля
                        stocks_to_portf_list.append(indx_max_sharp_ratio)
                        counter = counter + 1
                    else:
                        for j in range(0, columns_count):
                            if j in stocks_to_portf_list:    # Добавляю чтобы нельзя было повторно включать акции
                                sharp_ratio_list.append(0)
                            else:
                                share_prices = np.array(data[i-sharp_period:i,j], dtype=float)
                                sub_sumpl_comulative_income = comulative_income + share_prices 
                                income = np.sum(sub_sumpl_comulative_income)
                                std = np.std(sub_sumpl_comulative_income)
                                if std > 0 and income > 0:
                                    sharp_ratio = income/std
                                else:
                                    sharp_ratio = 0
                                sharp_ratio_list.append(sharp_ratio)
                        
                        max_sharp_ratio = max(sharp_ratio_list)
                        indx_max_sharp_ratio = np.argmax(sharp_ratio_list)
                        comulative_income = comulative_income + np.array(data[i-sharp_period:i,indx_max_sharp_ratio], dtype=float)
                        stocks_to_portf_list.append(indx_max_sharp_ratio)
                        counter = counter + 1
                
                stocks_to_portf_list = np.array(stocks_to_portf_list)
                for j in range(0,stocks_to_portf):
                    if j == 0:
                        stock_ind = stocks_to_portf_list[j]
                        results = np.array(data[i:i+hold,stock_ind])#, dtype=float)
                    else:
                        stock_ind = stocks_to_portf_list[j]
                        results = results + np.array(data[i:i+hold,stock_ind])#, dtype=float)
                results = np.array(results)        
                overal_results.append(results)
            
            overal_results = pd.DataFrame(overal_results)
            print(overal_results)
            overal_results = np.array(overal_results)    
            
            
            # Анализируем результаты
            print(overal_results.shape)
            strings = overal_results.shape[0]
            rows  = overal_results.shape[1]
            
            mean_profit_list = []
            P_L_list = []
            
            for i in range(0, overal_results.shape[0]):
                profit = 0
                loss = 0
                for j in range(0, overal_results.shape[1]):
                    if overal_results[i][j] >= 0:
                        profit = profit + overal_results[i][j]
                    else:
                        loss = loss + overal_results[i][j]
                        
                mean_profit = (profit + loss)/stocks_to_portf
                P_L = -profit / loss
                mean_profit_list.append(mean_profit)
                P_L_list.append(P_L)
            
            mean_profit_list = np.array(mean_profit_list)
            P_L_list = np.array(P_L_list)
            random_portf_list = np.array(random_portf_list)
            
             
            #print(max_sharp_ratio, indx_max_sharp_ratio)
            #print(stocks_to_portf_list)
            #print(mean_profit_list)
            mean_annual_profit = mean_profit_list.mean()*(250 / hold)     # Менять когда час 2250 или День 250 
            P_L = P_L_list.mean()
            random_income = random_portf_list.mean() * 250              # Менять когда час 2250 или День 250
            
            mean_annual_profit_by_hold.append(mean_annual_profit)
            P_L_by_hold.append(P_L)
            random_income_hold.append(random_income)
            
            print('mean_annual_profit:', mean_annual_profit)
            #print(P_L_list)
            print('P_L_list.mean:', P_L_list.mean())
            
            
            
            print('random_portf_mean_profit:', random_income)
            #random_mean_profit = data.mean() * 250
            #print('random_mean_profit:', random_mean_profit)
            
            
            #plt.plot(mean_profit_list)
            #plt.figure()
            #plt.plot(P_L_list)
        
        mean_annual_profit_by_hold = np.array(mean_annual_profit_by_hold)
        P_L_by_hold = np.array(P_L_by_hold)
        random_income_hold = np.array(random_income_hold)
        
        mean_annual_profit_overal.append(mean_annual_profit_by_hold)
        P_L_overal.append(P_L_by_hold)
        random_income_overal.append(random_income_hold)
        print('check')
            
    
    
mean_annual_profit_overal = np.array(mean_annual_profit_overal)       
вP_L_overal = np.array(P_L_overal)
random_income_overal = np.array(random_income_overal)
finish = datetime.now()
print('start: ',start)
print('finish: ',finish)
print('difference: ', finish-start)    

print('mean_annual_profit_overal:', mean_annual_profit_overal)
print('P_L_overal:', P_L_overal)

mean_annual_profit_overal = pd.DataFrame(mean_annual_profit_overal)
P_L_overal = pd.DataFrame(P_L_overal)
random_income_overal = pd.DataFrame(random_income_overal)

print('mean_annual_profit_overal:', mean_annual_profit_overal)
print('P_L_overal:', P_L_overal)
print('random_income_overal:', random_income_overal)

finish = datetime.now()
time_consumed = finish - start
print('start:', start)
print('finish:', finish)
print('time_consumed:', time_consumed)

#random_max_list = []
#random_min_list = []
#for i in range(0,random_array.shape[0]):
#    random_max_list.append(np.max(random_array[i,:]))
#    random_min_list.append(np.min(random_array[i,:]))
#
#print('random_max_list:', random_max_list)
#print('random_min_list:', random_min_list)

            0          1          2          3          4         5  \
0   -7.542581   0.585575 -11.592444   9.799239 -11.644156 -5.834131   
1    3.840300  -7.197346   5.413162   0.395239   1.832860  1.205487   
2   -2.937546  -3.989587  -2.823985   3.257777  -0.026802 -1.141512   
3   -2.626228   2.351052   1.049200  -8.083484  -2.334716  2.528948   
4   -1.120813  -1.598263  -6.435416  -8.829064   3.092247  3.844148   
5   -0.236070   0.738124   0.664561  -1.010357   0.330895  0.685328   
6    9.544798   3.200030   1.193530  -4.357578   6.383986 -0.214393   
7   -1.566634   9.936209  -3.032978  -8.842125  -4.166884  3.771262   
8   -2.684372   3.310343   0.856348  -5.242153   4.430195 -2.314373   
9    1.927763  -3.808341   4.223601   3.833471  -2.483938  5.469986   
10  -1.313787   3.862640   1.331588  -0.832527   3.608167 -1.806778   
11  -3.397585  -0.884442  -2.512548  -6.995091  -0.210693 -1.703118   
12  -0.715975   1.349347   3.243253  -0.920942  -2.296426  0.487261   
13  -4

          0         1          2          3          4         5          6   \
0  -7.542581  0.585575 -11.592444   9.799239 -11.644156 -5.834131  -0.742559   
1  -1.120813 -1.598263  -6.435416  -8.829064   3.092247  3.844148   3.159512   
2  -2.684372  3.310343   0.856348  -5.242153   4.430195 -2.314373  -3.431072   
3  -0.715975  1.349347   3.243253  -0.920942  -2.296426  0.487261   4.601411   
4  -3.958240 -3.410891   0.885732  -2.157758  -5.782520  3.557797  -2.996822   
5  -1.977815 -4.174708   0.396990   4.335668  -0.491887 -1.086154 -10.656961   
6  -1.173550  4.358344   2.707006   2.687873  -0.321946  0.314194   5.068997   
7   1.600529 -1.333851  -0.340919  -3.626125   0.741431 -3.480593   0.299885   
8  -4.133182 -4.462729  -1.129999  -1.186245  -0.195410  3.240003   0.307120   
9  -2.128441 -1.408904   2.742408  10.523483  -1.377650  1.518041   5.569516   
10 -3.510152 -3.215642   0.497778   0.802982  -0.602890 -5.523542   4.929704   

          7          8         9   ... 

         0         1          2          3          4         5         6   \
0 -7.542581  0.585575 -11.592444   9.799239 -11.644156 -5.834131 -0.742559   
1  1.927763 -3.808341   4.223601   3.833471  -2.483938  5.469986  4.752836   
2 -1.221702  4.105225   5.433458   3.129953   4.589990 -3.635176  5.184879   
3  0.589254  6.002244   3.636952  -1.788499   1.417327 -3.910981  0.300918   
4 -2.128441 -1.408904   2.742408  10.523483  -1.377650  1.518041  5.569516   

         7         8         9   ...         80        81        82        83  \
0  5.067851  9.700678  2.529642  ...   7.585801 -4.420995 -4.537752 -1.442774   
1  2.982432  0.278963 -4.531898  ... -11.142216  7.419966  5.629192 -5.358982   
2 -2.186836  4.577664  2.864735  ...  -2.083327  4.250797  1.577968  1.271708   
3 -4.155223  1.384071 -4.519110  ...   0.006156 -2.028419 -3.454477  0.136169   
4 -2.509308  0.972576 -5.698183  ...   4.367025  1.783574  1.821857 -2.157468   

         84        85        86        87   

        0         1          2         3          4         5         6    \
0 -7.542581  0.585575 -11.592444  9.799239 -11.644156 -5.834131 -0.742559   
1 -4.745735 -2.828265   5.028810 -3.418278   0.779354 -1.665938 -4.565607   
2 -4.165000 -2.674281   3.074585  2.611475   6.630753 -3.363566  3.689032   

        7         8         9    ...       160       161       162       163  \
0  5.067851  9.700678  2.529642  ... -4.846573 -7.590160 -1.773777 -5.301157   
1  1.381813 -4.457987 -0.221956  ... -3.432585  4.780260 -0.908536  0.139121   
2  8.422151  0.008262  8.916499  ... -3.804792 -2.493573  4.114452 -0.089703   

        164       165       166       167       168       169  
0 -0.308513  5.488066 -6.189773 -0.018719  6.023634 -1.708627  
1 -4.760493  1.232834 -7.390150  5.036903 -0.171066 -3.389383  
2  1.401031 -0.184304  1.340868 -3.900695  3.794730  1.580096  

[3 rows x 170 columns]
(3, 170)
mean_annual_profit: 6.952343631586518
P_L_list.mean: 1.08444194288544
random_port

          0         1         2         3         4         5         6   \
0  -0.815763 -2.553108 -2.040443  1.528377 -3.863780  1.592603 -0.978505   
1  -2.644920 -0.590477 -4.176224 -4.723712 -1.262059  5.737087  6.811301   
2   2.413952  1.385315  0.297041 -3.348178  0.752541 -2.374058 -1.734770   
3  -0.233979  0.249092  2.445143  1.449828 -1.676160  1.662717  2.526157   
4  -3.866727 -3.638164 -0.179945 -1.136161 -6.581512  6.438313 -2.576593   
5   0.276882 -2.925924 -1.953745  1.685476 -5.652868  0.178253 -7.783422   
6  -4.514154 -0.462451 -1.158740  3.111915 -1.817246  2.088690  3.771167   
7  -1.241605 -0.662659 -0.905835 -2.406325 -1.316729 -4.127220  1.345734   
8  -1.521874 -7.524172 -0.168395 -3.081055  3.600442  5.208512  1.699345   
9  -2.745328 -1.168432  0.796360  9.502832  0.622877  0.349854  2.456487   
10 -2.640350 -0.690131  1.168304  8.280529  3.408676  3.499930  2.900348   

          7          8         9   ...         30        31        32  \
0   2.938049  

...

          0         1         2         3         4         5         6   \
0  -2.605977  0.083898 -3.174034  1.989643 -5.843750 -4.273256  3.265714   
1   3.622991  2.003469 -3.426410 -2.530527  0.525747  2.141350 -1.983173   
2  -0.388791 -1.874627  2.223037  7.171888 -2.625621 -4.782728 -7.791195   
3 -10.391502 -2.430280  2.846031 -4.746824  3.996687 -3.620601  7.740501   
4  -6.490870  2.664718 -0.550664 -3.010594 -2.982643  2.290756  0.775174   

         7         8          9   ...        90        91        92        93  \
0  3.932732  1.985904   3.760568  ... -0.257770 -2.535280  2.784443 -1.393838   
1  4.344253 -5.582531  10.174884  ...  0.680427 -4.940953 -1.173392  7.166757   
2  7.096289 -4.049370  -6.652043  ...  2.506132 -0.336165  3.481238 -0.993921   
3 -2.740150  1.288262   2.591928  ... -2.703745 -8.188528 -0.862599 -3.217386   
4 -8.432846  1.364244  -1.760965  ... -0.128788 -5.106160 -2.448342  1.428252   

         94        95        96        97         98    

        0         1          2         3         4         5         6    \
0 -2.605977  0.083898  -3.174034  1.989643 -5.843750 -4.273256  3.265714   
1 -5.633499 -0.302940  10.981277 -5.970232 -6.390132 -1.013899 -4.548179   
2  1.237786 -4.432400   3.259123  2.507094  1.828605 -1.489969  2.926261   

        7         8         9    ...       160       161       162       163  \
0  3.932732  1.985904  3.760568  ... -0.643116  3.453690 -2.538965 -4.332329   
1 -0.571632 -1.136220 -1.417751  ... -5.009698  6.731837  2.001879  1.363834   
2  5.122440  1.418792  6.048880  ... -6.658828  0.435607  9.892783  1.444586   

        164       165       166       167       168       169  
0  0.110595  0.324797 -4.642132 -2.655385  0.090759 -4.457936  
1 -9.972954  5.158665 -1.910985  4.347809  0.518693 -2.200722  
2  2.165300 -2.925293  2.128927 -0.323609  0.432330  0.131153  

[3 rows x 170 columns]
(3, 170)
mean_annual_profit: 5.151522923060582
P_L_list.mean: 1.0728828546551499
random_portf_

          0         1          2          3          4         5         6   \
0  -2.605977  0.083898  -3.174034   1.989643  -5.843750 -4.273256  3.265714   
1   1.610818 -6.563393  -6.007642 -10.892232 -11.238940  3.528898  9.357902   
2   6.521867 -4.725489  -1.592937  -1.814267   3.764525  1.294058  0.404397   
3   2.752950 -1.449747   0.416175   1.324555  -1.946715  1.355519  5.249073   
4  -5.457967  6.004737  10.522654  -3.154154  -5.037269  9.667587 -0.713083   
5   0.566829 -0.877264   4.932530   4.184806   5.460722 -5.129147 -5.666418   
6  -5.191731  1.371626  -0.303126   6.868686  -5.700509  3.022039  4.541642   
7   1.250415 -1.965809   2.672123  -3.378703   3.455994 -8.046768  0.769877   
8  -2.126488 -2.204265   0.410950   0.942556  -1.112115  2.615359 -0.022286   
9  -1.714690  1.444257  -0.436015   8.581318   0.786753  2.671284 -0.233278   
10 -7.196090  3.229998   2.196416   0.977767  -1.190522  0.086531  0.864414   

          7          8         9   ...        30   

        0         1         2         3         4         5         6    \
0 -2.605977  0.083898 -3.174034  1.989643 -5.843750 -4.273256  3.265714   
1 -7.558294 -0.849880  1.081022 -3.803475  1.630603 -1.128100  0.739237   
2 -2.278122 -2.374761  8.702312 -3.430794 -1.740347  5.232131  0.196253   
3 -3.337491  5.324191  3.913085  0.919611 -4.721363  6.591168 -4.748224   

        7         8         9    ...       100       101       102       103  \
0  3.932732  1.985904  3.760568  ...  3.658221  1.542601 -0.391327  2.269051   
1 -2.042915 -2.482531  2.716727  ...  7.901811  3.039951 -0.174710  0.977301   
2  0.343999  0.586847 -1.799038  ... -2.183253 -2.108307  3.024196 -1.518131   
3  3.910269  1.908361 -0.914423  ... -0.316823 -4.345739 -4.350613  8.674305   

        104       105       106       107       108       109  
0  0.240123 -0.131594 -0.521583  0.974203  0.273601 -0.335506  
1  2.613593  0.743873  5.888900  1.129331  3.825224  2.704449  
2 -3.451043  3.241629  0.792076

14  12.299647  13.084305  16.165151  15.935842  16.539907  15.062869  
start: 2023-02-07 17:33:46.996574
finish: 2023-02-07 17:34:43.933541
time_consumed: 0:00:56.936967
